# Various Utilities
Lesser utilities that do not warrant their own script files.

In [1]:
from pathlib import Path
import sqlite3
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import statistics
import numpy as np

import time
import json
from pathlib import Path
import utilities as util

/home/jovyan/MasterThesis/mahjong_project/utilities/utilities.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange, tqdm


In [4]:
# !conda install -y -c conda-forge fastparquet python-snappy

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - fastparquet
    - python-snappy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    cramjam-2.3.1              |   py38h497a2fe_1         1.2 MB  conda-forge
    fastparquet-0.6.3          |   py38hb5d20a5_0         449 KB  conda-forge
    python-snappy-0.6.0        |   py38h49bdff1_0          31 KB  conda-forge
    thrift-0.13.0              |   py38h709712a_2         121 KB  conda-forge
    ------------------------------------------------------------
     

# Read State Data Parquets

In [5]:
STATE_LOG_PARQUET = Path.home() / 'MasterThesis' / 'data' / 'state_data_new' / '2018' / Path('2018031022gm-00a9-0000-58505c47.parquet')
STATE_LOG_PARQUET

PosixPath('/home/jovyan/MasterThesis/data/state_data_new/2018/2018031022gm-00a9-0000-58505c47.parquet')

In [6]:
with pd.option_context('display.max_rows', 2500, 'display.max_columns', 10, 'display.expand_frame_repr', True):
    display(pd.read_parquet(STATE_LOG_PARQUET, engine='fastparquet'))

FileNotFoundError: 

# Count Round Winds Occurrences

In [8]:
df_game = pd.read_parquet(Path('..') / "log_game_data.parquet", engine='fastparquet')
df_game = util.filter_logs(df_game)
df_game

,red,kui,ton-nan,sanma,soku,p0-dan,p1-dan,p2-dan,p3-dan,p0-rating,p1-rating,p2-rating,p3-rating
log_id,,,,,,,,,,,,,
2009022011gm-00a9-0000-d7935c6d,True,True,True,False,False,16,16,16,16,2096.00,2000.00,2030.00,2008.00
2009022020gm-00a9-0000-28418029,True,True,True,False,False,17,16,16,16,2043.00,2058.00,2024.00,2044.00
2009022021gm-00a9-0000-246550e3,True,True,True,False,False,16,16,16,17,2068.00,2020.00,2123.00,2095.00
2009022021gm-00a9-0000-3bacedde,True,True,True,False,False,17,16,16,16,2049.00,2121.00,2070.00,2092.00
2009022021gm-00a9-0000-4879b095,True,True,True,False,False,17,16,16,16,2102.00,2064.00,2028.00,2156.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019123123gm-00a9-0000-cd0c84d8,True,True,True,False,False,17,16,16,17,2208.53,2100.00,2163.85,2158.03
2019123123gm-00a9-0000-ceab322a,True,True,True,False,False,16,16,16,16,2151.90,2092.36,2081.30,2093.02
2019123123gm-00a9-0000-d9881584,True,True,True,False,False,17,16,17,17,2158.95,2122.33,2216.35,2166.84


In [9]:
df = pd.read_parquet(Path('..') / "log_round_meta.parquet", engine='fastparquet')
df

ParquetException: Metadata parse failed: ../log_round_meta.parquet

In [ ]:
df_game.index

In [23]:
df.loc[df_game.index]['round_wind'].value_counts()

0    8060303
1    6798549
2      93186
3          0
Name: round_wind, dtype: int64

In [15]:
df['round_wind'].value_counts()

0    13847051
1     9139649
2      126045
3          15
Name: round_wind, dtype: int64

# Find Kakans (Added Kan)

In [4]:
STATE_LOG_PATH = Path.home() / 'MasterThesis' / 'data' / 'state_data_new'
YEAR = 2018
LOG_RANGE_START = 0
LOG_RANGE_END   = 10000

kakans = []
for p in tqdm([l for l in (Path(STATE_LOG_PATH) / str(YEAR)).iterdir()][LOG_RANGE_START:LOG_RANGE_END], position=0):
    
    if len(kakans) > 10:
        break
    
    df = pd.read_parquet(p, engine='fastparquet')
    
    kakan_df =  df[
                (df['action'] == 'KaKan') | (df['action'] == 'MinKan')
    ]
    
    if len(kakan_df) > 0:
        kakans.append(kakan_df)

for c in kakans:
    row = c.iloc[0]
#     print("LOG ID:", row['log_id'], "\tROUND:", row['round'])
    with pd.option_context('display.max_rows', 1500, 'display.max_columns', 10, 'display.expand_frame_repr', True):
        display(c)

  0%|          | 30/10000 [00:00<05:08, 32.36it/s] 


,log_id,round,step,player,action,tile
889,2018031022gm-00a9-0000-58505c47,4,112,2,KaKan,108
890,2018031022gm-00a9-0000-58505c47,4,112,2,KaKan,109
891,2018031022gm-00a9-0000-58505c47,4,112,2,KaKan,110
892,2018031022gm-00a9-0000-58505c47,4,112,2,KaKan,111


,log_id,round,step,player,action,tile
1178,2018010115gm-00a9-0000-a2315941,6,52,2,KaKan,132
1179,2018010115gm-00a9-0000-a2315941,6,52,2,KaKan,133
1180,2018010115gm-00a9-0000-a2315941,6,52,2,KaKan,134
1181,2018010115gm-00a9-0000-a2315941,6,52,2,KaKan,135


,log_id,round,step,player,action,tile
290,2018062023gm-00a9-0000-74a3e1ed,1,94,1,KaKan,108
291,2018062023gm-00a9-0000-74a3e1ed,1,94,1,KaKan,109
292,2018062023gm-00a9-0000-74a3e1ed,1,94,1,KaKan,110
293,2018062023gm-00a9-0000-74a3e1ed,1,94,1,KaKan,111
1588,2018062023gm-00a9-0000-74a3e1ed,9,115,0,MinKan,76
1589,2018062023gm-00a9-0000-74a3e1ed,9,115,0,MinKan,77
1590,2018062023gm-00a9-0000-74a3e1ed,9,115,0,MinKan,78
1591,2018062023gm-00a9-0000-74a3e1ed,9,115,0,MinKan,79
1618,2018062023gm-00a9-0000-74a3e1ed,9,141,0,KaKan,80
1619,2018062023gm-00a9-0000-74a3e1ed,9,141,0,KaKan,81


,log_id,round,step,player,action,tile
1064,2018042410gm-00a9-0000-0f1e3d76,6,105,0,MinKan,64
1065,2018042410gm-00a9-0000-0f1e3d76,6,105,0,MinKan,65
1066,2018042410gm-00a9-0000-0f1e3d76,6,105,0,MinKan,66
1067,2018042410gm-00a9-0000-0f1e3d76,6,105,0,MinKan,67


...

,log_id,round,step,player,action,tile
281,2018071520gm-00a9-0000-8470a9cf,1,86,3,KaKan,8
282,2018071520gm-00a9-0000-8470a9cf,1,86,3,KaKan,9
283,2018071520gm-00a9-0000-8470a9cf,1,86,3,KaKan,10
284,2018071520gm-00a9-0000-8470a9cf,1,86,3,KaKan,11
608,2018071520gm-00a9-0000-8470a9cf,3,84,0,KaKan,44
609,2018071520gm-00a9-0000-8470a9cf,3,84,0,KaKan,45
610,2018071520gm-00a9-0000-8470a9cf,3,84,0,KaKan,46
611,2018071520gm-00a9-0000-8470a9cf,3,84,0,KaKan,47


,log_id,round,step,player,action,tile
2052,2018020617gm-00a9-0000-8321e6bb,13,60,2,KaKan,120
2053,2018020617gm-00a9-0000-8321e6bb,13,60,2,KaKan,121
2054,2018020617gm-00a9-0000-8321e6bb,13,60,2,KaKan,122
2055,2018020617gm-00a9-0000-8321e6bb,13,60,2,KaKan,123


,log_id,round,step,player,action,tile
462,2018040804gm-00a9-0000-59e2a9bb,2,90,3,KaKan,80
463,2018040804gm-00a9-0000-59e2a9bb,2,90,3,KaKan,81
464,2018040804gm-00a9-0000-59e2a9bb,2,90,3,KaKan,82
465,2018040804gm-00a9-0000-59e2a9bb,2,90,3,KaKan,83


,log_id,round,step,player,action,tile
941,2018121912gm-00a9-0000-a301a0ac,6,69,1,MinKan,76
942,2018121912gm-00a9-0000-a301a0ac,6,69,1,MinKan,77
943,2018121912gm-00a9-0000-a301a0ac,6,69,1,MinKan,78
944,2018121912gm-00a9-0000-a301a0ac,6,69,1,MinKan,79
1245,2018121912gm-00a9-0000-a301a0ac,8,42,1,KaKan,108
1246,2018121912gm-00a9-0000-a301a0ac,8,42,1,KaKan,109
1247,2018121912gm-00a9-0000-a301a0ac,8,42,1,KaKan,110
1248,2018121912gm-00a9-0000-a301a0ac,8,42,1,KaKan,111
1299,2018121912gm-00a9-0000-a301a0ac,8,92,1,KaKan,120
1300,2018121912gm-00a9-0000-a301a0ac,8,92,1,KaKan,121


,log_id,round,step,player,action,tile
1145,2018122923gm-00a9-0000-3b6d9e3c,6,92,2,KaKan,0
1146,2018122923gm-00a9-0000-3b6d9e3c,6,92,2,KaKan,1
1147,2018122923gm-00a9-0000-3b6d9e3c,6,92,2,KaKan,2
1148,2018122923gm-00a9-0000-3b6d9e3c,6,92,2,KaKan,3


# Find Chankans (Robbing a Kan)

In [ ]:
STATE_LOG_PATH = "E:/mahjong/state_data/"
YEAR = 2019
LOG_RANGE_START = 0
LOG_RANGE_END   = 10000

chankans = []
for p in tqdm([l for l in (Path(STATE_LOG_PATH) / str(YEAR)).iterdir()][LOG_RANGE_START:LOG_RANGE_END], position=0):
    
    df = pd.read_parquet(p)
    
    chankan_df =  df[
                ((df['action'] == 'KaKan') & (df['action'].shift(-1) == 'Remove')) | 
                ((df['action'] == 'Remove') & (df['action'].shift(1) == 'KaKan')) |
                ((df['action'] == 'Ron') & (df['action'].shift(1) == 'Remove') & (df['action'].shift(2) == 'KaKan'))
    ]
    
    if len(chankan_df) > 0:
        chankans.append(chankan_df)

for c in chankans:
    row = c.iloc[0]
    print("LOG ID:", row['log_id'], "\tROUND:", row['round'])
#     with pd.option_context('display.max_rows', 1500, 'display.max_columns', 10, 'display.expand_frame_repr', True):
#         display(c)

# Count occurences of calls

```
dict format: 
{ mjlog_ID: 
    [
        p0: {'Chi': 0, 'Pon': 0, 'Kan': 0}
        p1: {'Chi': 0, 'Pon': 0, 'Kan': 0}
        p2: {'Chi': 0, 'Pon': 0, 'Kan': 0}
        p3: {'Chi': 0, 'Pon': 0, 'Kan': 0}
    ],
    [
        p0: {'Chi': 0, 'Pon': 0, 'Kan': 0}
        p1: {'Chi': 0, 'Pon': 0, 'Kan': 0}
        p2: {'Chi': 0, 'Pon': 0, 'Kan': 0}
        p3: {'Chi': 0, 'Pon': 0, 'Kan': 0})
    ],
    ...
}
```

In [ ]:
JSON_LOG_PATH = "E:/mahjong/logs_json/"

for year in Path(JSON_LOG_PATH).iterdir():
    
    calls_counts = {}  # See above log format
    
    all_logs = [ll for ll in year.iterdir()]

    for log in tqdm(all_logs, total=len(all_logs), desc=year.stem):
        
        calls_counts[log.stem] = []
        
        log_dict = json.loads(log.read_text())

        for game_round in log_dict['rounds']:

            _round = [{'Chi': 0, 'Pon': 0, 'MinKan': 0, 'KaKan': 0, 'AnKan': 0, 'Nuki': 0} for _ in range(4)]

            for action in game_round:
                if action['tag'] == 'CALL':
                    player = action['data']["caller"]
                    call_type = action['data']["call_type"]
                    _round[player][call_type] += 1

            calls_counts[log.stem].append(_round)
    
    # Convert dict to pandas then to .feather file
    rows = []
    for log_id, rounds in tqdm(calls_counts.items()):
        for round_num, round_info in enumerate(rounds):
            row = {"log_id": log_id, "round_number": round_num}
            for player_id, player_calls in enumerate(round_info):
                for call_type, call_counts in player_calls.items():
                    row[f'p{player_id}-{call_type}'] = call_counts
            # {'Chi': 0, 'Pon': 0, 'MinKan': 0, 'KaKan': 0, 'AnKan': 0, 'Nuki': 0} for _ in range(4)

            rows.append(row)
    
df = pd.DataFrame(rows)

# Find Largest number of discards done by players
Careful! This is finding the highest number of discards given a player! As players can call on player's discards, and therefore taking the discarded tile away from the pool.

In other words: 

**A player's discard pool is a subset of a player's discards**. 

In [ ]:
years = [
    '2009',
    '2010',
    '2011',
    '2012',
    '2013',
    '2014',
    '2015',
    '2016',
    '2017',
    '2018',
    '2019',
]

rows = []

path = Path("E:/mahjong/logs_json/")
for year in path.iterdir():
    
    if year.stem not in years:
        continue

    all_logs = [ll for ll in year.iterdir()]

    for log in tqdm(all_logs, total=len(all_logs), desc=year.stem):

        log_json = json.load(log.open())

        for rn, _round in enumerate(log_json['rounds']):

            discard_counts = [0, 0, 0, 0]
            for action in _round:
                if action['tag'] == 'DISCARD':
                    player = action['data']['player']
                    discard_counts[player] += 1

            row = {
                'log_id': log.stem,
                 'round': rn,
                    'p0': discard_counts[0],
                    'p1': discard_counts[1],
                    'p2': discard_counts[2],
                    'p3': discard_counts[3]
            }

            rows.append(row)

In [ ]:
df = pd.DataFrame(rows)
df

In [ ]:
df.to_feather(Path('D:') / 'accumulated_pandas' / f"better_discard_counts.feather")

In [10]:
game_data = pd.read_parquet(Path('E:') / 'mahjong' / 'pandas' / 'log_game_data.parquet')

FileNotFoundError: 

In [ ]:
# Retrieve years the logs are from
game_data.insert(0, 'year', game_data.index.str[:4].astype(int))
game_data

## Filtered Version

In [ ]:
filtered_game_data = util.filter_logs(game_data)
filtered_game_data

# Number of Logs per Year

In [ ]:
num_year_logs = game_data.year.value_counts().sort_index().rename('Original')  # Original
num_logs_filtered = filtered_game_data.year.value_counts().sort_index().rename('Filtered')  # Filtered

num_log_df = pd.DataFrame([num_year_logs, num_logs_filtered]).T

display(num_log_df)
print("Original Total:", num_log_df['Original'].sum())
print("Filtered Total:", num_log_df['Filtered'].sum())

In [ ]:
log_plot = num_log_df.plot.bar(rot=0)

# Game Configs Distribution

In [15]:
for i in (Path('..') / 'log_game_data.parquet').iterdir():
    print(i)

NotADirectoryError: [Errno 20] Not a directory: '../log_game_data.parquet'

In [58]:
game_data = pd.read_parquet(Path('..') / 'log_game_data.parquet')

config_distribution = game_data[['red', 'kui', 'soku', 'ton-nan', 'sanma']].value_counts(sort=True, normalize=True)

cdf = config_distribution.to_frame().reset_index()
cdf[0] *= 100
cdf[0] = cdf[0].map('{:,.3f}'.format)
# cdf.replace({True: 'On', False: 'Off'})

In [60]:
game_data

,red,kui,ton-nan,sanma,soku,p0-dan,p1-dan,p2-dan,p3-dan,p0-rating,p1-rating,p2-rating,p3-rating
log_id,,,,,,,,,,,,,
2009020103gm-00a9-0000-2453a04c,True,True,True,False,False,0,0,0,0,1500.00,1500.00,1509.00,1500.00
2009020103gm-00a9-0000-47e70b77,True,True,True,False,False,0,0,0,0,1500.00,1500.00,1500.00,1500.00
2009022011gm-00a9-0000-d7935c6d,True,True,True,False,False,16,16,16,16,2096.00,2000.00,2030.00,2008.00
2009022011gm-00e1-0000-2820118d,True,True,False,False,True,17,16,16,17,2011.00,2032.00,2090.00,2070.00
2009022011gm-00e1-0000-293fb785,True,True,False,False,True,16,16,17,17,2034.00,2089.00,2004.00,2068.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019123123gm-00e1-0000-99d83b7a,True,True,False,False,True,16,16,16,16,2061.34,2187.98,2075.11,2122.54
2019123123gm-00e1-0000-c92676e9,True,True,False,False,True,16,16,16,17,2034.50,2085.77,2074.37,2179.28
2019123123gm-00e1-0000-cf8b36ed,True,True,False,False,True,18,16,17,16,2240.79,2036.80,2172.86,2072.47


In [59]:
print(cdf.to_latex(index=False))

\begin{tabular}{llllll}
\toprule
   red &    kui &   soku &  ton-nan &  sanma &       0 \\
\midrule
  True &   True &  False &     True &  False &  55.635 \\
  True &   True &   True &    False &  False &  22.320 \\
  True &   True &  False &     True &   True &  21.557 \\
  True &   True &  False &    False &   True &   0.196 \\
 False &   True &  False &     True &  False &   0.086 \\
  True &   True &   True &     True &  False &   0.075 \\
  True &   True &  False &    False &  False &   0.064 \\
  True &   True &   True &     True &   True &   0.037 \\
  True &   True &   True &    False &   True &   0.018 \\
 False &  False &  False &    False &  False &   0.009 \\
 False &   True &  False &    False &  False &   0.002 \\
 False &  False &  False &     True &  False &   0.001 \\
\bottomrule
\end{tabular}



In [17]:
ans = [pd.DataFrame(y) for x, y in game_data.groupby('year', as_index=False)]
for a in ans:
    display(a)

KeyError: 'year'

# Various Plotting

In [ ]:
game_data

## Average Table Rating
We calculate the mean of players' ratings for each log.

In [ ]:
player_rating_columns = ['p0-rating', 'p1-rating', 'p2-rating', 'p3-rating']
average_table_ratings = filtered_game_data[player_rating_columns].mean(axis=1)
average_table_ratings.reset_index(drop=True, inplace=True)
average_table_ratings

In [ ]:
print("Average Table Rating: {:>8.2f}".format(statistics.mean(average_table_ratings)))
print("Median Table Rating:  {:>8.2f}".format(statistics.median(average_table_ratings)))
print("Max Table Rating:     {:>8.2f}".format(max(average_table_ratings)))
print("Min Table Rating:     {:>8.2f}".format(min(average_table_ratings)))

### Plotting

In [ ]:
# Plot Params
## X-Axis
min_rating = 2000
max_words = 2300
shift = 4
tick_shift = 100

## Y-Axis
max_reviews = 110000
y_shift = 10000

In [ ]:
cm = plt.cm.get_cmap('plasma')  # Other noteworthy ones: 'plasma', 'Spectral'

plt.figure(figsize=(10, 5))
ax = plt.gca()
plt.grid(axis='y', linestyle='--', alpha=0.7)
ax.set_axisbelow(True)
ax.tick_params(axis='x', rotation=0)
n, bins, patches = plt.hist(average_table_ratings, 
                            bins=range(min_rating, max_words, shift),
#                             density=True, 
                            edgecolor='black', 
                            linewidth=1.2
                           )

for i, p in enumerate(patches):
        plt.setp(p, 'facecolor', cm(i/55))  # Change this value to diversify color


plt.title("Rating Distribution (2009-2019)", fontsize=17)
plt.xticks(np.arange(min_rating, max_words + tick_shift, tick_shift))
plt.yticks(np.arange(0, max_reviews + y_shift, y_shift))
plt.ylim(0, max_reviews)

plt.xlabel("Average Table Rating", fontsize=15)
plt.ylabel("No. of Games", fontsize=15)
# plt.savefig(imdb_graphs_output / 'IMDB_length_unprocessed_en', bbox_inches='tight')
plt.show()

print("Number of occurence not shown:", sum((x > max_words or x < min_rating) for x in average_table_ratings))

# Discard Pile Size Plot (FIX - REMOVE SANMA GAMES?)

In [ ]:
discard_count_df = pd.read_feather(Path('E:') / 'mahjong' / 'pandas' / 'discard_counts.feather')
discard_count_df = discard_count_df[discard_count_df.log_id.isin(filtered_game_data.index)]  # Filterblade
discard_count_df

In [ ]:
max_discard = discard_count_df[['p0-discards', 'p1-discards', 'p2-discards', 'p3-discards']].max(axis=1)
max_pool = discard_count_df[['p0-pool', 'p1-pool', 'p2-pool', 'p3-pool']].max(axis=1)

max_discard.max(), max_pool.max()

In [ ]:
all_players_discards = discard_count_df[['p0-discards', 'p1-discards', 'p2-discards', 'p3-discards']].stack().reset_index(drop=True)
all_players_pool     = discard_count_df[['p0-pool', 'p1-pool', 'p2-pool', 'p3-pool']].stack().reset_index(drop=True)

In [ ]:
total_game_discards = discard_count_df[['p0-discards', 'p1-discards', 'p2-discards', 'p3-discards']].sum(axis=1).value_counts().sort_index()

In [ ]:
discard_count_df['total_discards'] = discard_count_df[['p0-discards', 'p1-discards', 'p2-discards', 'p3-discards']].sum(axis=1)
discard_count_df['total_pool_size'] = discard_count_df[['p0-pool', 'p1-pool', 'p2-pool', 'p3-pool']].sum(axis=1)
discard_count_df

In [ ]:
discard_count_df.total_discards.max(), discard_count_df.total_pool_size.max()

# Games and their maximum Phase
The following code proves that some rounds do not reach mid game or late game.
Note: This is round-wise count, whereas when we train our models, we use state-based count. However, this metrics are worth writing about:

In [ ]:
discard_count_df.total_discards.groupby(pd.cut(discard_count_df.total_discards, (0, 24, 48, 100))).count()

In [ ]:
discard_count_df.total_pool_size.groupby(pd.cut(discard_count_df.total_pool_size, (0, 24, 48, 100))).count()

In [ ]:
# Plot Params
## X-Axis
min_rating = 0
max_words = 82
tick_shift = 2

## Y-Axis
max_reviews = 2_500_000
y_shift = 500_000

In [ ]:
accumulated_df = pd.DataFrame({"Discards": discard_count_df.total_discards.value_counts().sort_index(), "Pool": discard_count_df.total_pool_size.value_counts().sort_index()})

In [ ]:
accumulated_df.plot.bar(rot=0, stacked=False, figsize=(15, 5), subplots=True, edgecolor='black', linewidth=1, legend=True, 
                        title='Discard Vs Pool Distribution',
                        sharex=False,
                        sharey=True,
                        xticks=np.arange(min_rating, max_words + tick_shift, tick_shift),
                        yticks=np.arange(0, max_reviews + y_shift, y_shift),
                        xlabel="No. of Discards",
                        ylabel='No. of Rounds',
                        colormap='Spectral',
                        fontsize=10
#                         grid=True
                       )


# plt.figure(figsize=(20, 5))
# ax = plt.gca()
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# ax.set_axisbelow(True)
# ax.tick_params(axis='x', rotation=0)

# plt.bar(list(discard_count_df.total_discards.value_counts(sort=False).index), 
#         list(discard_count_df.total_discards.value_counts(sort=False)), 
#         align='center',
#         edgecolor='black', 
#         linewidth=1.2
# )

# plt.bar(list(discard_count_df.total_pool_size.value_counts(sort=False).index), 
#         list(discard_count_df.total_pool_size.value_counts(sort=False)), 
#         align='center',
#         edgecolor='black', 
#         linewidth=1.2
# )

# plt.title("Total Discards Distribution", fontsize=17)

# plt.grid(axis='y', linestyle='--', alpha=0.7)
# ax = plt.gca()
# ax.set_axisbelow(True)

# plt.xticks(np.arange(min_rating, max_words + tick_shift, tick_shift))
# plt.yticks(np.arange(0, max_reviews + y_shift, y_shift))
# plt.ylim(0, max_reviews)

# plt.xlabel("No. of Discards", fontsize=15)
# plt.ylabel("No. of Rounds", fontsize=15)
# plt.savefig(imdb_graphs_output / 'IMDB_length_unprocessed_en', bbox_inches='tight')
# plt.gca().ticklabel_format(axis='both', style='plain')

plt.show()

In [ ]:
cm = plt.cm.get_cmap('Spectral')  # Other noteworthy ones: 'plasma', 'Spectral'

plt.figure(figsize=(10, 5))
ax = plt.gca()
plt.grid(axis='y', linestyle='--', alpha=0.7)
ax.set_axisbelow(True)
ax.tick_params(axis='x', rotation=0)

plt.bar(list(all_players_discards.value_counts(sort=False).index), 
        list(all_players_discards.value_counts(sort=False)), 
        align='center',
        edgecolor='black', 
        linewidth=1.2
)

plt.bar(list(all_players_pool.value_counts(sort=False).index), 
        list(all_players_pool.value_counts(sort=False)), 
        align='center',
        edgecolor='black', 
        linewidth=1.2
)

plt.title("Max Discard Pile Size Distribution", fontsize=17)
plt.xticks(np.arange(min_rating, max_words + tick_shift, tick_shift))
plt.yticks(np.arange(0, max_reviews + y_shift, y_shift))
plt.ylim(0, max_reviews)

plt.xlabel("Max Discard Pile Size", fontsize=15)
plt.ylabel("No. of Rounds", fontsize=15)
# plt.savefig(imdb_graphs_output / 'IMDB_length_unprocessed_en', bbox_inches='tight')
plt.gca().ticklabel_format(axis='both', style='plain')

plt.show()

# print("Number of Players:", players)
# print("Number of occurence not shown:", sum((x > max_words or x < min_rating) for x in average_table_ratings))